# Оцениваем регрессии на данных HS0 в спецификациях:
- Сырые данные ~ ковид и ограничения + контроль на дамми
- Сырые данные ~ ковид и ограничения + контроль на дамми | инструментальные переменные

Для второй спецификации считаем бутстраповские доверительные интервалы

In [1]:
library(arrow)
library(stargazer)
library(tidyverse)
library(plotly)
library(fixest)
signif.code = signif.code = c("***" = 0.001, "**" = 0.01, "*" = 0.05, "." = 0.10)


Присоединяю пакет: 'arrow'


Следующий объект скрыт от 'package:utils':

    timestamp



Please cite as: 


 Hlavac, Marek (2022). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.3. https://CRAN.R-project.org/package=stargazer 


Warning message:
"пакет 'tidyverse' был собран под R версии 4.4.2"
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ lubridate::duration() masks arrow::duration()
✖ dplyr::filter()       masks stats::filter()
✖ dplyr::lag()          masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Warning message:
"пакет 'plotly' был собран под R версии 4.4.2"

Присоеди

## Сырые данные

Под каждой регрессией вывожу summary с кластеризацией ошибок. В конце блока сравниваю робастность результата по используемым ошибкам.

In [2]:
# Скачиваю данные и переназываю переменные
df = read_parquet('../data/processed_data/trade_hs0_x_covid.parquet.gzip')
# Переменная pair нужна, чтобы считать НАС стандартные ошибки
df = df %>% mutate(pair = paste(i, j))

df$deaths_i = df$deaths
df$deaths_j = df$`deaths:r_j`

df$median_age_i = df$median_age
df$median_age_j = df$`median_age:r_j`

df$average_surface_temperature_i = df$average_surface_temperature
df$average_surface_temperature_j = df$`average_surface_temperature:r_j`

df$C1M_School_closing_i = df$C1M_School_closing
df$C1M_School_closing_j = df$`C1M_School_closing:r_j`

df$population_density_i = df$population_density
df$population_density_j = df$`population_density:r_j`

df$StringencyIndex_Average_i = df$StringencyIndex_Average
df$StringencyIndex_Average_j = df$`StringencyIndex_Average:r_j`

df$share_1_members_i = df$share_1_members
df$share_1_members_j = df$`share_1_members:r_j`

df$H6M_Facial_Coverings_i = df$H6M_Facial_Coverings
df$H6M_Facial_Coverings_j = df$`H6M_Facial_Coverings:r_j`

df$C8EV_International_travel_controls_i = df$C8EV_International_travel_controls
df$C8EV_International_travel_controls_j = df$`C8EV_International_travel_controls:r_j`
df %>% head()

t,i,j,v,q,v19,q19,country,t:r_i,cases,⋯,population_density_i,population_density_j,StringencyIndex_Average_i,StringencyIndex_Average_j,share_1_members_i,share_1_members_j,H6M_Facial_Coverings_i,H6M_Facial_Coverings_j,C8EV_International_travel_controls_i,C8EV_International_travel_controls_j
<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2021,LUX,LBY,968.018,210.173,501.858,127.012,LUX,2021,241.1512329,⋯,248.620682,4.055137,46.04334,63.24680,33.34,NA,2.0082192,2.827397,2.534247,2.520548
2021,RUS,SEN,1307165.125,3131615.500,789625.305,2232100.632,RUS,2021,138.1677534,⋯,8.838488,89.445110,46.46488,24.90279,25.69,6.92,2.3150685,3.402740,2.610959,2.561644
2020,SOM,KOR,3136.611,2045.876,5608.268,4002.770,SOM,2020,0.7276796,⋯,26.542530,531.109006,34.65910,49.42956,1.53,31.75,0.7622951,1.653005,2.101093,2.415301
2020,TUR,CRI,57792.285,98850.453,60858.013,86163.873,TUR,2020,67.6935912,⋯,108.343854,98.596161,54.57148,54.39423,6.37,12.07,2.2732241,1.218579,2.040984,2.322404
2021,KNA,BIH,0.520,0.008,0.460,0.005,KNA,2021,164.5090411,⋯,179.857692,63.377090,NA,38.78658,NA,11.50,NA,2.476712,NA,1.000000
2022,AZE,MLT,22060.938,39007.156,12476.490,33493.400,AZE,2022,56.3560822,⋯,122.707272,1660.971875,49.01625,22.13003,6.77,22.64,0.6575342,2.323288,3.000000,1.263014


In [11]:
# FE (т.е. оценка на панельных данных с использованием внутригрупповой дисперсии)
fe_no_iv = feols(log(v/v19) ~ log(deaths_i) + log(deaths_j) +
#  C8EV_International_travel_controls_i + C8EV_International_travel_controls_j + 
 StringencyIndex_Average_i + StringencyIndex_Average_j| i + j + year,
 data = df)
fe_no_iv %>% summary(cluster = ~ i+j)

NOTE: 13,568 observations removed because of NA and infinite values (RHS: 13,568).



OLS estimation, Dep. Var.: log(v/v19)
Observations: 56,739
Fixed-effects: i: 175,  j: 175,  year: 3
Standard-errors: Clustered (i & j) 
                           Estimate Std. Error   t value Pr(>|t|)    
log(deaths_i)             -0.011330   0.005792 -1.956073 0.052058 .  
log(deaths_j)              0.001151   0.005069  0.227106 0.820608    
StringencyIndex_Average_i -0.001548   0.000818 -1.893591 0.059939 .  
StringencyIndex_Average_j -0.001718   0.000774 -2.219336 0.027757 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 0.98151     Adj. R2: 0.038389
                Within R2: 3.764e-4

In [12]:
# Добавляю гетерогенность эффектов во времени
heterogenous_fe = feols(log(v/v19) ~ year/log(deaths_i) + year/log(deaths_j) +
# year/C8EV_International_travel_controls_i + year/C8EV_International_travel_controls_j + 
year/StringencyIndex_Average_i + year/StringencyIndex_Average_j|
i + j ,
data = df)
heterogenous_fe %>% summary(cluster = ~ i+j)

NOTE: 13,568 observations removed because of NA and infinite values (RHS: 13,568).



OLS estimation, Dep. Var.: log(v/v19)
Observations: 56,739
Fixed-effects: i: 175,  j: 175
Standard-errors: Clustered (i & j) 
                                    Estimate Std. Error   t value   Pr(>|t|)
year2021                           -0.066473   0.082709 -0.803699 0.42266720
year2022                           -0.093258   0.106148 -0.878564 0.38084958
year2020:log(deaths_i)             -0.001376   0.006490 -0.212105 0.83227342
year2021:log(deaths_i)             -0.002369   0.006645 -0.356522 0.72188158
year2022:log(deaths_i)             -0.015950   0.006663 -2.393687 0.01774359
year2020:log(deaths_j)             -0.004274   0.006617 -0.645889 0.51920234
year2021:log(deaths_j)              0.002303   0.007058  0.326238 0.74463663
year2022:log(deaths_j)              0.006998   0.006529  1.071799 0.28529483
year2020:StringencyIndex_Average_i -0.004905   0.001341 -3.656185 0.00033886
year2021:StringencyIndex_Average_i -0.002618   0.001041 -2.514227 0.01283687
year2022:StringencyIndex_Av

### Инструментальные переменные на панельных данных

In [13]:
# Спецификация с фиксированными эффектами, куда я добавил инструменты, чтобы убрать потенциальную двустороннюю связь. 

fe_iv = feols(log(v/v19) ~  1 +
#  C8EV_International_travel_controls_i + C8EV_International_travel_controls_j +
 StringencyIndex_Average_i + StringencyIndex_Average_j
 | year+i+j|
 log(deaths_i) + log(deaths_j) ~ 
# H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
# +
C1M_School_closing_i+C1M_School_closing_j
+
median_age_i + median_age_j
+
population_density_i + population_density_j
# +
# average_surface_temperature_i + average_surface_temperature_j # Добавляет пропусков

 ,
 data = df
 )
fe_iv %>% summary(cluster = ~ i+j)

NOTE: 13,906 observations removed because of NA and infinite values (RHS: 9,802, IV: 6,067/10,661).



TSLS estimation - Dep. Var.: log(v/v19)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(v/v19)
Observations: 56,401
Fixed-effects: year: 3,  i: 175,  j: 175
Standard-errors: Clustered (i & j) 
                           Estimate Std. Error   t value Pr(>|t|)    
fit_log(deaths_i)         -0.061580   0.030578 -2.013830 0.045568 *  
fit_log(deaths_j)          0.055663   0.037004  1.504240 0.134332    
StringencyIndex_Average_i -0.000471   0.001066 -0.441394 0.659476    
StringencyIndex_Average_j -0.002799   0.001199 -2.335368 0.020665 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 0.982993     Adj. R2:  0.034575
                 Within R2: -0.003635
F-test (1st stage), log(deaths_i): stat = 344.5    , p < 2.2e-16 , on 6 and 56,044 DoF.
F-test (1st stage), log(deaths_j): stat = 3

### Экспорт результатов в латех

In [14]:
etable(fe_no_iv, heterogenous_fe, fe_iv, vcov = ~i+j, signif.code=signif.code, fitstat=~ . + sargan.p + wh.p + ivf, tex=FALSE)

,,fe_no_iv,heterogenous_fe,fe_iv
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(v/v19),log(v/v19),log(v/v19)
2,,,,
3,log(deaths_i),-0.0113. (0.0058),,-0.0616* (0.0306)
4,log(deaths_j),0.0012 (0.0051),,0.0557 (0.0370)
5,StringencyIndex_Average_i,-0.0015. (0.0008),,-0.0005 (0.0011)
6,StringencyIndex_Average_j,-0.0017* (0.0008),,-0.0028* (0.0012)
7,year2021,,-0.0665 (0.0827),
8,year2022,,-0.0933 (0.1061),
9,year2020 x log(deaths_i),,-0.0014 (0.0065),


In [15]:
etable(fe_no_iv, heterogenous_fe, fe_iv, vcov = ~i+j, signif.code=signif.code, fitstat=~ . + sargan.p + wh.p + ivf, tex=TRUE)

\begingroup
\centering
\begin{tabular}{lccc}
   \tabularnewline \midrule \midrule
   Dependent Variable: & \multicolumn{3}{c}{log(v/v19)}\\
   Model:                                           & (1)           & (2)             & (3)\\  
   \midrule
   \emph{Variables}\\
   log(deaths\_i)                                   & -0.0113$^{.}$ &                 & -0.0616$^{*}$\\   
                                                    & (0.0058)      &                 & (0.0306)\\   
   log(deaths\_j)                                   & 0.0012        &                 & 0.0557\\   
                                                    & (0.0051)      &                 & (0.0370)\\   
   StringencyIndex\_Average\_i                      & -0.0015$^{.}$ &                 & -0.0005\\   
                                                    & (0.0008)      &                 & (0.0011)\\   
   StringencyIndex\_Average\_j                      & -0.0017$^{*}$ &                 & -0.0028$^{*}$\\   
         